# 问题：预测信用卡欺诈 

## 业务场景简介
您在一家跨国银行工作。在过去的几个月里，遭遇信用卡欺诈的客户数量显著增加。一家大型新闻媒体最近甚至发表了一篇关于您和其他银行正在经历的信用卡欺诈的报道。

面对这一情况，您需要在欺诈性信用卡交易给公司造成更严重的影响之前，利用机器学习来识别这类交易，从而解决部分问题。您已经获得了对过去信用卡交易数据集的访问权限，您可以使用该数据集训练一个机器学习模型，来预测交易是否属于欺诈性交易。


## 关于该数据集
该数据集包含欧洲持卡人在 2013 年 9 月通过信用卡进行的交易。该数据集展示了两天内发生的交易，包括欺诈性交易和合法交易的样本。

### 特征
该数据集包含超过 30 个数值特征，由于数据的个人隐私问题，大部分特征都经过了主成分分析 (PCA) 转换。未经过 PCA 转换的特征只有 'Time' 和 'Amount'。特征 'Time' 包含数据集中每笔交易与第一笔交易之间经过的秒数。特征 'Amount' 是交易金额。'Class' 是响应或目标变量，如果存在欺诈，则取值 '1'，否则取值 '0'。

特征： 
`V1, V2, ...V28`：使用 PCA 获得的主成分

非 PCA 特征：
- `Time`：数据集中每笔交易与第一笔交易之间经过的秒数，$T_x - t_0$
- `Amount`：交易金额；此特征可用于基于样本的代价敏感学习 
- `Class`：目标变量，如果存在欺诈，则 `Fraud = 1`，否则 `Not Fraud = 0`

### 数据集属性
网站：https://www.openml.org/d/1597

Twitter：https://twitter.com/dalpozz/status/645542397569593344

作者：Andrea Dal Pozzolo、Olivier Caelen 和 Gianluca Bontempi
来源：信用卡欺诈检测 – 2015 年 6 月 25 日
官方引文：Andrea Dal Pozzolo、Olivier Caelen、Reid A. Johnson 和 Gianluca Bontempi。Calibrating Probability with Undersampling for Unbalanced Classification（利用欠采样解决不平衡分类问题来实现概率校准）。在 2015 年 IEEE 计算智能和数据挖掘 (CIDM) 学术研讨会上的发言。

该数据集是由 Worldline 公司和比利时布鲁塞尔大学 (ULB) 机器学习小组 (mlg.ulb.ac.be) 在有关大数据挖掘和欺诈检测的研究合作期间进行收集和分析的。有关相关主题的当前和过去项目的更多详细信息，请访问 http://mlg.ulb.ac.be/BruFence 和 http://mlg.ulb.ac.be/ARTML。

# 步骤 1：问题定义和数据收集

在开始这个项目时，先在下面用几句话来总结在这个场景中要解决的业务问题和要实现的业务目标。同时说明您想让自己的团队衡量的业务指标。确定这些信息后，请清楚地写出机器学习问题陈述。最后，添加一两条备注，说明问题陈述所对应的机器学习的类型。

#### <span style="color: blue;">项目演示：在项目演示中总结这些详细信息。</span>

### 通读业务场景，然后：

### 1.确定 ML 是否是适合部署的解决方案以及原因。

In [ ]:
# 在此处输入答案

### 2.定义业务问题、成功指标和期望的 ML 输出。

In [ ]:
# 在此处输入答案

### 3.确定您要解决的 ML 问题的类型。

In [ ]:
# 在此处输入答案

### 4.分析您要使用的数据是否合适。

In [ ]:
# 在此处输入答案

### 设置

我们已经确定了工作重点，现在我们来进行一些设置，以便开始解决问题。

**注意：**该笔记本是在 `ml.m4.xlarge` 笔记本实例上创建和测试的。

In [ ]:
# 导入各种 Python 库

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time

sns.set()
%matplotlib inline

In [ ]:
# 安装 imblearn
!pip uninstall scikit-learn -y
!pip install imbalanced-learn==0.5.0
!pip install imblearn

### 下载数据集

In [ ]:
# 检查文件是否已位于所需路径中，或者是否需要下载
# 数据源：https://www.openml.org/data/get_csv/1673544/phpKo8OWT
import os
import subprocess
base_path = '/home/ec2-user/SageMaker/project/data/FraudDetection'
file_path = '/fraud.csv'

if not os.path.isfile(base_path + file_path):
    subprocess.run(['mkdir', '-p', base_path])
    subprocess.run(['aws', 's3', 'cp',
                    's3://aws-tc-largeobjects/ILT-TF-200-MLDWTS/credit_card_project/',
                    base_path,'--recursive'])
else:
    print('File already downloaded!')

# 步骤 2：数据预处理和可视化  
在数据预处理阶段，您应该利用机会探索数据并将其可视化，以便更好地了解数据。首先，导入必要的库并将数据读入 Pandas dataframe。然后，探索数据。查看数据集的形状，探索要使用的列及其类型（数值型、分类型）。对各项特征执行基本统计，以便了解特征的均值和范围；深入分析目标列并确定其分布。

### 要考虑的特定问题
1.您可以从对特征执行的基本统计中推断出什么？ 

2.您可以从目标类的分布中推断出什么？

3.您从数据探索中还推断出了什么？

#### <span style="color: blue;">项目演示：在项目演示中总结这些问题和其他类似问题的答案。</span>

将 CSV 数据读入 Pandas DataFrame您可以使用内置的 Python `read_csv` 函数（[文档](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html)）。

In [ ]:
df = pd.read_csv(<CODE>) # 在此处输入代码

通过打印数据集的前 5 行，检查一下 DataFrame。 

**提示**：使用 `<dataframe>.head()` 函数。

In [ ]:
# 在此处输入代码

In [ ]:
# 该类有一个奇怪的字符串，而不是一个布尔值或数字 0 和 1，所以要将它转换为 0 和 1 

mapped_class = {"'0'": 0, "'1'": 1}
df['Class'] = df['Class'].map(lambda x: mapped_class[x])

In [ ]:
# 检查是否可行

df.head()

**任务**：验证数据集中的所有列，看看它们是否就是您在上文读取的内容：`V1-V28`、`Time`、`Amount` 和 `Class`。 

**提示**：使用 `<dataframe>.columns` 检查 DataFrame 中的列。

In [ ]:
# 在此处输入代码

In [ ]:
# 在此处输入答案

**问题**：关于列类型和空值，您可以发现什么？ 有多少个列是数值型或分类型？ 

**提示**：使用 `info()` 函数来检查。

In [ ]:
# 在此处输入代码

**问题**：使用 Pandas 库和 `Describe` 函数执行基本统计。`amount` 特征的平均值和标准偏差是多少？ 您可以从这些数字中推断出什么？

In [ ]:
# 在此处输入代码

In [ ]:
# 在此处输入答案

**问题**：对于存在欺诈性的记录，它们的 `amount` 的平均值、标准偏差和最大值是多少？  

**提示**：使用 DataFrame 中内置的 `mean()`、`std()` 和 `max()` 函数。

In [ ]:
print("Fraud Statistics")

avg_amt = # 在此处输入代码
std_dev = # 在此处输入代码
max_amt = # 在此处输入代码

print(f"The average amount is {avg_amt}")
print(f"The std deviation for amount is {std_dev}")
print(f"The max amount is {max_amt}")

In [ ]:
# 在此处输入答案

下面来看一下目标变量 `Class`。首先，您可以看到该变量的分布情况。
 
**问题**：什么是类的分布？  

**提示**：使用 `<dataframe>['column_name'].value_counts()` 查看分布。

In [ ]:
# 在此处输入代码

In [ ]:
# 在此处输入答案

**问题**：您可以从类的分布中推断出什么？

In [ ]:
# 在此处输入答案：

**问题**：获得 0 值的类的数量与记录总数的比率是多少？

In [ ]:
# 在此处输入代码

## 可视化数据
如果您未在上面部分执行此操作，您可以使用下面的空间来进一步可视化某些数据。特别要看一下 `Amount` 和 `Time` 等特征的分布情况，还要计算一下数据集中各特征之间的线性相关性。

### 要考虑的特定问题
1.查看 `Amount` 和 `Time` 等特征的分布情况之后，这些特征对您的模型可能会有多大的帮助？ 您可以通过分布情况推断出哪些可能有助于您更好地理解数据的信息？

2.当您只看标记为“欺诈”的数据时，`Amount` 和 `Time` 等特征的分布情况是否不同？

3.数据集中有没有任何特征是紧密相关的？ 如果有，那么您接下来的步骤是什么？

使用下方的单元格可视化您的数据，并回答这些问题以及您可能感兴趣的其他问题。根据需要插入和删除单元格。

#### <span style="color: blue;">项目演示：在项目演示中总结这些问题以及类似问题的答案。</span>

从简单的散点图开始。绘制 V1 与V2 的图。有关绘制散点图的更多信息，请参阅 [Matplotlib 文档](https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.plot.html)。

In [ ]:
plt.plot(<CODE>, <CODE>, '.') # 在此处输入代码

查看一些特征的分布情况。使用 `sns.distplot()` 查看 `Amount` 和 `Time` 等单个特征的分布情况。

In [ ]:
sns.distplot(<CODE>) # 在此处输入代码

**问题**：`Time` 特征对您有什么帮助？ 再次查看分布情况。您可以从散点图中推断出什么？

In [ ]:
# 在此处输入答案：

绘制直方图和核密度估计 (KDE) 图。

In [ ]:
sns.distplot(df['Time'])

In [ ]:
sns.distplot(df[df['Class'] == 1]['Time'])

**问题**：对于 `Amount` 列中的欺诈案例，它的分布是什么样的？

In [ ]:
sns.distplot(<CODE>)# 在此处输入代码

下面我们来看一下使用名为 `pairplot` 的 Seaborn 函数展示的分布情况。`pairplot` 可创建一个散点图网格，这样数据集中的每个特征都是一次用作 X 轴，一次用作 Y 轴。此网格的对角线显示了该特征的数据分布情况。

查看 `V1`、`V2`、`V2`、`V4`和 `Class` 的散点图矩阵。您在矩阵中看到了什么？ 您能否从这些特征中区分欺诈和非欺诈？  

**提示**：创建一个新的 DataFrame，包含 `V1`、`V2`、`V4` 和 `Class` 列。

In [ ]:
new_df = # 在此处输入代码
sns.pairplot(new_df, hue="Class")

您可以看到，对于我们使用的小型特征子集，有一种方法可以区分欺诈和非欺诈，但是要根据任何一个特征来区分它们就不容易了。

下面我们来看看这些特征是如何相互作用的。使用 Pandas `<dataframe>.corr()` 函数计算数据集的所有特征之间的线性相关性。对相关性进行可视化总是更容易一些。使用 Seaborn heatmap (`sns.heatmap`) 函数绘制相关性图，将 `annot` 标记设置为 `True`。

In [ ]:
plt.figure(figsize = (25,15))
correlation_matrix = # 在此处输入代码
sns.heatmap(correlation_matrix, annot=True,fmt=".2f")

**问题**：对于相关的特征，您应该在进行模型训练之前删除其中一个特征。您有没有发现任何可以删除的特征？  

In [ ]:
# 在此处输入答案：

## <span style="color:red">实验 2 结束</span>

将项目文件保存到本地计算机。按照以下步骤进行操作：

1.在页面顶部，单击 **File（文件）**菜单。

1.选择 **Download as（另存为）**，然后单击 **Notebook (.ipynb)**。 

这会将当前的笔记本下载到计算机上默认的下载文件夹中。

# 步骤 3：模型训练和评估

在将数据集从 DataFrame 转换为机器学习算法可以使用的格式之前，您必须完成一些预备步骤。对于 Amazon SageMaker，您需要采取以下步骤：

1.使用 `sklearn.model_selection.train_test_split` 将数据拆分为 `train_data`、`validation_data` 和 `test_data`。   
2.将数据集转换为适合 Amazon SageMaker 训练作业使用的文件格式。可以是 CSV 文件或记录 protobuf。有关更多信息，请参阅 [用于训练的常见数据格式](https://docs.aws.amazon.com/sagemaker/latest/dg/cdf-training.html)。   
3.将数据上传到 Amazon S3 存储桶中。

使用以下单元格完成这些步骤。根据需要插入和删除单元格。

#### <span style="color: blue;">项目演示：在项目演示中记下您在此阶段做出的关键决定。</span>



- 要用于训练和模型数据的 Amazon Simple Storage Service (Amazon S3) 存储桶和前缀 (?)。它们应与笔记本实例、训练和托管位于同一区域。
- 用于提供数据访问权限以便进行训练和托管的 AWS Identity and Access Management (IAM) 角色 [Amazon 资源名称 (ARN)] (https://docs.aws.amazon.com/general/latest/gr/aws-arns-and-namespaces.html)。请参阅文档，了解如何创建这些角色。

**注意：**如果笔记本实例、训练和/或托管需要不止一个角色，请用相应的完整 IAM 角色的 ARN 字符串替换 `get_execution_role（）` 调用。

用实验账户提供的资源名称替换 **`<LabBucketName>`**。

In [ ]:
import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.amazon.amazon_estimator import RecordSet

# 实例化 Amazon SageMaker 会话
sess = sagemaker.Session()

# 获取 Amazon SageMaker 角色 
role = get_execution_role()

# 存储桶名称
bucket = <LabBucketName>

# 获取包括线性学习器算法的容器的映像 URI
container = get_image_uri(boto3.Session().region_name, 'linear-learner')

print (f'Session {sess} ')
print(f'The role is {role}')
print(f'The container is {role} in the {boto3.Session().region_name} region')

In [ ]:
from sklearn.model_selection import train_test_split

def create_training_sets(data):
    """
    将 DataFrame 转换为训练、验证和测试
    参数：
        data：要拆分数据集的 DataFrame
    返回值：
        train_features：训练特征数据集
        test_features：测试特征数据集 
        train_labels：训练数据集的标签
        test_labels：测试数据集的标签
        val_features：验证特征数据集
        val_labels：验证数据集的标签
    """
    # 从 DataFrame 提取目标变量并将类型转换为 float32
    ys = np.array(<CODE>).astype("float32") # 在此处输入代码
    
    # 删除所有不需要的列，包括目标列
    drop_list = # 在此处输入代码
    
    # 删除 drop_list 中的列并将数据转换为 float32 类型的 NumPy 数组
    xs = np.array(data.drop(<CODE>, axis=1)).astype("float32")# 在此处输入代码
    
    np.random.seed(0)

    # 使用 sklearn 函数 train_test_split 按训练 80% 和测试 20% 的比例拆分数据集
    # 示例：train_test_split(x, y, test_size=0.3)
    train_features, test_features, train_labels, test_labels = # 在此处输入代码
    
    # 再次使用 sklearn 函数将测试数据集拆分为 50% 用作验证和 50% 用作测试
    val_features, test_features, val_labels, test_labels = # 在此处输入代码
    
    return train_features, test_features, train_labels, test_labels, val_features, val_labels

In [ ]:
# 使用函数来创建您的数据集
train_features, test_features, train_labels, test_labels, val_features, val_labels = create_training_sets(df)

print(f"Length of train_features is: {<CODE>}")
print(f"Length of train_labels is: {<CODE>}")
print(f"Length of val_features is: {<CODE>}")
print(f"Length of val_labels is: {<CODE>}")
print(f"Length of test_features is: {<CODE>}")
print(f"Length of test_labels is: {<CODE>}")

### 示例输出
```
Length of train_features is: (227845, 29)  
Length of train_labels is: (227845,)  
Length of val_features is: (28481, 29)  
Length of val_labels is: (28481,)  
Length of test_features is: (28481, 29)  
Length of test_labels is: (28481,)  
```

### 模型训练

首先，通过一个 ml.m4.xlarge 实例来实例化具有 `predictor_type='binary_classifier'` 参数的线性学习器评估程序。

In [ ]:
import sagemaker
from sagemaker.amazon.amazon_estimator import RecordSet
import boto3

# 实例化线性学习器评估程序对象
num_classes = # 在此处输入代码

# 通过一个 ml.m4.xlarge 实例来实例化线性学习器评估程序“二元分类器”对象
linear = sagemaker.LinearLearner(role=sagemaker.get_execution_role(),
                                               train_instance_count=<CODE>,
                                               train_instance_type=<CODE>,
                                               predictor_type=<CODE>)

### 示例代码
```
num_classes = len(pd.unique(train_labels))
linear = sagemaker.LinearLearner(role=sagemaker.get_execution_role(),
                                              train_instance_count=1,
                                              train_instance_type='ml.m4.xlarge',
                                              predictor_type='binary_classifier',
                                             )
                                              
```

线性学习器接受内容类型为 protobuf 或 CSV 的训练数据以及内容类型为 protobuf、CSV 或 JSON 的推理请求。训练数据有特征和 ground-truth 标签，而推理请求中的数据只有特征。在生产管道中，建议将数据转换为 Amazon SageMaker protobuf 格式并将其存储在 Amazon S3 中。但是，为了快速启动并运行，AWS 能够在数据集小到足以装入本地内存时提供便捷的 `record_set` 方法来进行转换和上传。该方法接受您已有的 NumPy 数组，所以我们在这里就用它。`RecordSet` 对象将会跟踪数据的临时 Amazon S3 位置。使用 `estimator.record_set` 函数来创建训练、验证和测试记录。然后使用 `estimator.fit` 函数启动训练作业。

In [ ]:
### 创建训练、验证和测试记录
train_records = linear.record_set(<CODE>,<CODE>, channel='train')# 在此处输入代码
val_records = linear.record_set(<CODE>,<CODE>, channel='validation')# 在此处输入代码
test_records = linear.record_set(<CODE>,<CODE>, channel='test')# 在此处输入代码

下面，我们来使用您上传的数据集训练模型。

### 示例代码
```
linear.fit([train_records,val_records,test_records], wait=True, logs='All')
```

In [ ]:
### 安装分类器
# 在此处输入代码

## 模型评估
在本节中，您将评估自己训练的模型。首先，使用 `estimator.deploy` 函数（`initial_instance_count= 1` 且 `instance_type= 'ml.m4.xlarge'`）在 Amazon SageMaker 上部署模型。

In [ ]:
linear_predictor = linear.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

现在您已经有一个托管的终端节点在运行了，您可以通过提出 HTTP POST 请求，轻松地利用模型进行实时预测。但首先，您需要设置序列化器和解串器，以便将 `test_features` NumPy 数组传递给终端节点背后的模型。您还要计算模型的混淆矩阵，以便评估它针对测试数据所实现的可视化效果。

In [ ]:
from sklearn.metrics import accuracy_score,precision_score, recall_score
#from sagemaker.predictor import csv_serializer, json_deserializer, numpy_deserializer
#from sagemaker.predictor import csv_deserializer

def predict_batches(model, features, labels, split=200):
    """
    预测通过拆分指定的批量中的数据点。
    将数据拆分为 <split> 个分段并对每个分段调用 
    model.predict
    参数：
        model：您将用于调用预测函数的模型
        features：预测依据的数据集
        labels：记录的 true 值
        split：将数据拆分成的分段数量
    返回值：
        无
    """

    split_array = np.array_split(features, split)
    predictions = []
    for array in split_array:
        predictions += model.predict(array).label

    # preds = np.array([p['predicted_label'] for p in predictions])
    preds = [i.label['predicted_label'].float32_tensor.values[0] for i in predictions]
    
    # 计算准确率accuracy = accuracy_score(labels, preds)
    
    print(f'Accuracy: {accuracy}')
    
    # 计算查准率
    precision = precision_score(labels, preds)
    print(f'Precision: {precision}')
    
    # 计算查全率
    recall = recall_score(labels, preds)
    print(f'Recall: {recall}')
    
    confusion_matrix = pd.crosstab(index=labels, columns=np.round(preds), rownames=['True'], colnames=['predictions']).astype(int)
    plt.figure(figsize = (5,5))
    sns.heatmap(confusion_matrix, annot=True, fmt='.2f', cmap="YlGnBu").set_title('Confusion Matrix') 
    

现在，您的终端节点是 'InService'，请评估模型在测试集上的表现。将测试集上的表现与训练集上的表现进行比较。

### 要考虑的关键问题：
1.您的模型在测试集上的表现与训练集上的表现相比如何？ 您可以从这种比较中推断出什么？ 

2.准确率、查准率和查全率等指标结果之间有没有明显差异？ 如果是，为什么您会看到这些差异？ 

3.考虑到您的业务状况和目标，您此时最需要考虑的指标是什么？ 为什么？

4.您认为最重要指标的结果是否足以满足您的业务需求？ 如果不能，那么您在下一次迭代中可能会更改哪些内容（在特征设计部分，下一步是什么）？ 

用下面的单元格来回答这些问题和其他问题。根据需要插入和删除单元格。

#### <span style="color: blue;">项目演示：在项目演示中记录这些问题的答案以及您可能会在本节中回答的其他类似问题的答案。在项目演示中记录关键的详细信息和您所做的关键决定。</span>

In [ ]:
predict_batches(linear_predictor, test_features, test_labels)

与测试集一样，您也可以查看训练集的指标。注意，您还可以在上述的日志中看到这些指标。

In [ ]:
#predict_batches(linear_predictor, train_features, train_labels)

In [ ]:
# 删除推理终端节点
sagemaker.Session().delete_endpoint(linear_predictor.endpoint)


## <span style="color:red"> 实验 3 结束</span>

将项目文件保存到本地计算机。按照以下步骤进行操作：

1.在页面顶部，单击 **File（文件）**菜单。

1.选择 **Download as（另存为）**，然后单击 **Notebook (.ipynb)**。 

这会将当前的笔记本下载到计算机上默认的下载文件夹中。

# 迭代 II

# 步骤 4：特征设计

现在，您已经完成了一次训练和评估模型的迭代。鉴于模型第一次获得的结果可能不足以解决您的业务问题，您可以对数据进行哪些更改以改进模型的表现？

### 要考虑的关键问题：
1.两个主要类（欺诈与非欺诈）的平衡可能会如何影响模型性能？
2.平衡数据集是否会对特征之间的相关性产生任何影响？
3.在此阶段，您是否可以执行可能会对模型性能产生积极影响的特征约简技术？ 
4.执行一些特征设计之后，模型性能与第一次迭代相比如何？

使用下面的单元格执行您认为可以提高模型性能的特定特征设计技术（根据上述问题）。根据需要插入和删除单元格。

#### <span style="color: blue;">项目演示：在项目演示中记录您在本节中使用的主要决策和方法，以及在再次评估模型后获得的任何新性能指标。</span>

在开始之前，思考一下为什么查准率和查全率在 80% 左右，而准确率为 99%。

In [ ]:
# 在此处输入答案

准确率是根据模型预测正确的样本数计算而来的。但是，大多数样本实际上是阴性样本，所以如果在这个非常不平衡的数据集中预测所有样本为 0，您仍然可以得到大概 99.827% 的准确率。拥有一个不平衡数据集可能会导致一些算法性能问题。因此，在训练模型之前先处理数据的不平衡问题会很有帮助。

**问题**：您如何解决数据集不平衡的问题？


In [ ]:
# 在此处输入答案

**问题**：再次打印您的数据集形状。

In [ ]:
print(f"Length of train_features is: {train_features.shape}")
print(f"Length of train_labels is: {train_labels.shape}")
print(f"Length of val_features is: {val_features.shape}")
print(f"Length of val_labels is: {val_labels.shape}")
print(f"Length of test_features is: {test_features.shape}")
print(f"Length of test_labels is: {test_labels.shape}")

使用 `sns.countplot` 绘制数据集的原始分布。

In [ ]:
sns.countplot(df['Class'])
plt.title('Original Distribution of the dataset')

将 `train_features` 转换回 DataFrame。

In [ ]:
df_train = pd.DataFrame(<CODE>, columns = df.columns.drop(['Time','Class'])) # 在此处输入代码
df_train['Target'] = # 在此处输入代码

In [ ]:
df_train.head()

处理不平衡数据集的方式主要有两种：

- 过采样，增加更多阳性样本
    - 随机过采样
    - [合成少数类过采样技术 (SMOTE)](https://arxiv.org/abs/1106.1813)
- 欠采样，减少阴性样本
    - 随机欠采样
    - 使用聚类方法生成中心点

您可以使用一个名为 `Imbalanced-learn` 的库对数据集进行采样。`imbalanced-learn` 是一个 Python 程序包，提供多种重采样技术，通常用于存在严重类间不平衡的数据集。它与 scikit-learn 兼容，是 scikit-learn-contrib 项目的一部分。有关更多信息，请参阅 [imbalanced-learn API 文档](https://imbalanced-learn.org/stable/introduction.html)。

对于此示例，首先选择欠采样方法。要创建平衡数据集，请执行以下步骤：

1.使用所有阳性样本创建新的 DataFrame `fraud_df`。
2.创建另一个 DataFrame `non_fraud_df` 并使用 `dataframe.sample`，使其数值与 `fraud_df` DataFrame 和 `random_state=235` 相同。
3.将两个 DataFrame 合并成一个新的 DataFrame `balanced_df`。

In [ ]:
# 选择 df_train DataFrame 中 Target == 1 的行
fraud_df = # 在此处输入代码

# 选择 df_train DataFrame 中 Target == 0 的行
non_fraud_df = # 在此处输入代码

balanced_df = pd.concat([fraud_df, non_fraud_df], ignore_index=True, sort=False)

balanced_df.head()

使用 `sns.countplot()` 再次查看分布和形状。

In [ ]:
# 在此处输入代码
plt.title('Original Distribution of the dataset')

In [ ]:
balanced_df.shape

在进行训练之前，看一下如果您对数据集使用 T-分布随机近邻嵌入 (t-SNE) 等特征约简技术，会发生什么。

In [ ]:
from sklearn.manifold import TSNE

X_embedded = TSNE(n_components=2).fit_transform(<CODE>)
X_embedded.shape

In [ ]:
from matplotlib.colors import ListedColormap
plt.figure(figsize = (10,10))
plt.scatter(X_embedded[:,0], X_embedded[:,1],
            c = balanced_df['Target'],
            s = 1,
            cmap = ListedColormap(['Red', 'Blue']),
            linewidths=1)

plt.title('Red: 0 , Blue: 1')

**问题**：t-SNE 能否帮助您区分欺诈与非欺诈？  

In [ ]:
# 在此处输入答案

现在您已经有了新数据，下面来比较一下前后的相关性矩阵。

In [ ]:
# 确保使用子样本计算相关性

plt.figure(figsize = (20,10))

# 使用原始数据集查找特征之间的相关性
correlation_matrix_before = # 在此处输入代码
sns.heatmap(correlation_matrix_before, annot=True,fmt=".2f")

plt.figure(figsize = (20,10))

# 使用原始数据集查找特征之间的相关性
correlation_matrix_after = # 在此处输入代码
sns.heatmap(correlation_matrix_after, annot=True,fmt=".2f")

**问题**：通过查看不同的相关性矩阵，您可以推断出什么？ 如果您发现了差异，您能否分析出存在差异的原因？

In [ ]:
# 在此处输入答案

**问题**：您是否会因相关的数据而删除任何列？

In [ ]:
# 在此处输入答案

由于存在一些相关性，我们删除相关性超过 0.9 的相关数据。运行下面的单元格，删除 `V17` 和 `V18` 列。

In [ ]:
balanced_df_drop = balanced_df.drop(columns=['V17','V18'])

现在，可以使用新的平衡数据集进行训练、部署和评估了。

In [ ]:
# 在此处输入代码

### 示例代码

```
# 实例化线性学习器评估程序对象
num_classes = len(pd.unique(train_labels))
linear_estimator_balanced = sagemaker.LinearLearner(role=sagemaker.get_execution_role(),
                                               train_instance_count=1,
                                               train_instance_type='ml.m4.xlarge',
                                               predictor_type='binary_classifier')


train_records_bal = linear_estimator_balanced.record_set(balanced_df.drop(['Target'], axis=1).as_matrix(),
                                                balanced_df['Target'].as_matrix(),
                                                channel='train')
val_records_bal = linear_estimator_balanced.record_set(val_features, val_labels, channel='validation')
test_records_bal = linear_estimator_balanced.record_set(test_features, test_labels, channel='test')

linear_estimator_balanced.fit([train_records_bal, val_records_bal, test_records_bal])
```

减少能够实现分布均匀的样本数量导致了查全率下降，而不是上升。我们来尝试一下不同的策略，因为我们需要较高的查全率。

尝试使用 SMOTE 增加阳性样本的数量。

In [ ]:
from imblearn.over_sampling import SMOTE 

# 删除原始数据集中不需要的列
X = # 在此处输入代码

# 使用类特征作为标签
y = # 在此处输入代码

sm = SMOTE(random_state=35)
X_res, y_res = sm.fit_resample(X, y)

**可选**：将新数据集转换为 Pandas DataFrame 并查看数据的形状和分布。

In [ ]:
smote_df = pd.DataFrame(<CODE>, # 在此处输入代码
                        columns = df.drop(['Class', 'Time'], axis=1).columns) 
smote_df['Class'] = # 在此处输入代码
smote_df['Time'] = df['Time']

创建新的培训、测试和验证数据集。

In [ ]:
train_features, test_features, train_labels, test_labels, val_features, val_labels = create_training_sets(<CODE>))# 在此处输入代码

使用新的数据集训练模型。

In [ ]:
num_classes = len(pd.unique(train_labels))
linear_estimator_smote = sagemaker.LinearLearner(role=sagemaker.get_execution_role(),
                                               train_instance_count=1,
                                               train_instance_type='ml.m4.xlarge',
                                               predictor_type='binary_classifier')


train_records_smote = linear_estimator_smote.record_set(train_features, train_labels, channel='train')
val_records_smote = linear_estimator_smote.record_set(val_features, val_labels, channel='validation')
test_records_smote = linear_estimator_smote.record_set(test_features, test_labels, channel='test')

linear_estimator_smote.fit([train_records_smote, val_records_smote, test_records_smote])

**问题**：通过评估训练作业，您可以推断出什么？  

In [ ]:
# 在此处输入答案

### 超参数优化
模型优化阶段的另一部分是执行超参数优化。本节为您提供了一个机会来优化您的超参数，看看优化能在多大程度上提高模型性能。使用以下模板代码帮助您开始启动 Amazon SageMaker 超参数优化作业并查看评估指标。通过以下问题来帮助您完成本节的其余部分。

### 要考虑的关键问题：
1.随着优化作业时间的增加，您所选目标指标的结果会有何变化？ 您能得到的不同目标指标和时间之间有何关系？ 
2.目标指标和各个超参数之间的相关性是什么？ 是否存在与目标指标有强相关性的超参数？ 如果有，您可能会做些什么来利用这一强相关性？
3.分析超参数优化后的模型性能。当前的性能是否足以满足您解决业务问题的需求？

#### <span style="color: blue;">项目演示：在项目演示中记录您在本节中使用的主要决策和方法，以及在再次评估模型后获得的任何新性能指标。</span>

In [ ]:
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner

hyperparameter_ranges = {'wd': ContinuousParameter(<CODE>, <CODE>),
                        'l1': ContinuousParameter(<CODE>, <CODE>),
                        'learning_rate': ContinuousParameter(<CODE>, <CODE>)
                        }

objective_metric_name = <CODE>

tuner = HyperparameterTuner(<ENTER your estimator name>,
                            objective_metric_name,
                            hyperparameter_ranges,
                            max_jobs=10,
                            max_parallel_jobs=3)

tuner.fit([<CODE>], include_cls_metadata=False)

### 跟踪超参数优化作业进度

启动优化作业后，您可以通过调用 `describe_tuning_job` API 查看作业进度。`describe_tuning_job` 的输出是一个 JSON 对象，它包含有关优化作业当前状态的信息。您可以调用 `list_training_jobs_for_tuning_job` 来查看优化作业所启动的训练作业的详细列表。

In [ ]:
client = boto3.Session().client('sagemaker')
tuning_job_result = client.describe_hyper_parameter_tuning_job(HyperParameterTuningJobName=tuner.latest_tuning_job.job_name)

status = tuning_job_result['HyperParameterTuningJobStatus']
while status != 'Completed':
    print('Reminder: the tuning job has not been completed.')
    
    job_count = tuning_job_result['TrainingJobStatusCounters']['Completed']
    print("%d training jobs have completed" % job_count)
    
    time.sleep(180)

    tuning_job_result = client.describe_hyper_parameter_tuning_job(HyperParameterTuningJobName=tuner.latest_tuning_job.job_name)
    status = tuning_job_result['HyperParameterTuningJobStatus']
    
print("\n\n All training jobs have completed")
is_minimize = (tuning_job_result['HyperParameterTuningJobConfig']['HyperParameterTuningJobObjective']['Type'] != 'Maximize')
objective_name = tuning_job_result['HyperParameterTuningJobConfig']['HyperParameterTuningJobObjective']['MetricName']

In [ ]:
from pprint import pprint
if tuning_job_result.get('BestTrainingJob',None):
    print("Best model found so far:")
    pprint(tuning_job_result['BestTrainingJob'])
else:
    print("No training jobs have reported results yet.")

### 将所有结果提取为 DataFrame

您可以列出所有训练作业的超参数和目标指标，然后选取具有最佳目标指标的训练作业。

In [ ]:
tuner_analytics = sagemaker.HyperparameterTuningJobAnalytics(tuner.latest_tuning_job.job_name)

full_df = tuner_analytics.dataframe()

if len(full_df) > 0:
    df = full_df[full_df['FinalObjectiveValue'] > -float('inf')]
    if len(df) > 0:
        df = df.sort_values('FinalObjectiveValue', ascending=is_minimize)
        print("Number of training jobs with valid objective: %d" % len(df))
        print({"lowest":min(df['FinalObjectiveValue']),"highest": max(df['FinalObjectiveValue'])})
        pd.set_option('display.max_colwidth', -1) # 请勿截断 TrainingJobName        
    else:
        print("No training jobs have reported valid results yet.")
        
df


**问题**：模型优化有用吗？  

In [ ]:
# 在此处输入答案

### 查看优化作业结果与时间

接下来，您将展示随着优化作业的进行，目标指标会如何随时间而变化。对于贝叶斯策略，您将看到一个趋向于更好结果的总体趋势，但这种进展会不稳定，因为算法需要在探索新的参数空间领域与探索已知的良好领域之间实现平衡。这可以让您了解训练作业的数量是否足以应对搜索空间的复杂性。

In [ ]:
import bokeh
import bokeh.io
bokeh.io.output_notebook()
from bokeh.plotting import figure, show
from bokeh.models import HoverTool

class HoverHelper():

    def __init__(self, tuning_analytics):
        self.tuner = tuning_analytics

    def hovertool(self):
        tooltips = [
            ("FinalObjectiveValue", "@FinalObjectiveValue"),
            ("TrainingJobName", "@TrainingJobName"),
        ]
        for k in self.tuner.hyperparameter_ranges().keys():
            tooltips.append( (k, "@{%s}" % k) )

        ht = HoverTool(tooltips=tooltips)
        return ht

    def tools(self, standard_tools='pan,crosshair,wheel_zoom,zoom_in,zoom_out,undo,reset'):
        return [self.hovertool(), standard_tools]

hover = HoverHelper(tuner)

p = figure(plot_width=900, plot_height=400, tools=hover.tools(), x_axis_type='datetime')
p.circle(source=df, x='TrainingStartTime', y='FinalObjectiveValue')
show(p)

### 分析目标指标和各个超参数之间的相关性

现在，您已经完成了一个优化作业，您可能想知道目标指标和您选择要优化的各个超参数之间的相关性。拥有这一见解将帮助您确定，是否可以调整某些超参数的搜索范围并启动另一个优化作业。例如，如果您发现目标指标和某个数值型超参数之间存在正相关趋势，那么您可能需要在下一个优化作业中为该超参数设置一个更高的优化范围。

以下单元格将为每个超参数绘制一个图表，来显示它们与目标指标之间的相关性。

In [ ]:
ranges = tuner_analytics.tuning_ranges
figures = []
for hp_name, hp_range in ranges.items():
    categorical_args = {}
    if hp_range.get('Values'):
        # 这被标记为分类型。检查所有选项是否都是数值。
        def is_num(x):
            try:
                float(x)
                return 1
            except:
                return 0           
        vals = hp_range['Values']
        if sum([is_num(x) for x in vals]) == len(vals):
            # Bokeh 无法绘制实际为数值的“分类型”范围，所以就绘制数值型范围print("Hyperparameter %s is tuned as categorical, but all values are numeric" % hp_name)
            
        else:
            # 设置额外选项，用于绘制分类型范围。如果它们实际上是数值，会有点棘手。
            categorical_args['x_range'] = vals

    # 现在开始绘制
    p = figure(plot_width=600, plot_height=600,
               title="Objective vs %s" % hp_name,
               tools=hover.tools(),
               x_axis_label=hp_name, y_axis_label=objective_name,
               **categorical_args)
    p.circle(source=df, x=hp_name, y='FinalObjectiveValue')
    figures.append(p)
show(bokeh.layouts.Column(*figures))

将此部署为最终模型并使用测试集对其进行评估。

In [ ]:
tuned_model_deploy = tuner.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

In [ ]:
predict_batches(tuned_model_deploy, test_features, test_labels)

In [ ]:
predict_batches(tuned_model_deploy, val_features, val_labels)

### 可选：尝试 XGBoost 算法
接下来进入训练阶段，首先我们需要指定 XGBoost 算法容器的位置。

In [ ]:
containers = {'us-west-2': '433757028032.dkr.ecr.us-west-2.amazonaws.com/xgboost:latest',
              'us-east-1': '811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest',
              'us-east-2': '825641698319.dkr.ecr.us-east-2.amazonaws.com/xgboost:latest',
              'eu-west-1': '685385470294.dkr.ecr.eu-west-1.amazonaws.com/xgboost:latest'}


bucket = sess.default_bucket()
prefix = 'sagemaker/xgboost-creditcard'

from sagemaker.amazon.amazon_estimator import get_image_uri
container = get_image_uri(boto3.Session().region_name, 'xgboost')

然后，由于您使用 CSV 文件格式进行训练，请创建 s3_input，训练函数可以用它来指向 Amazon S3 中的文件。

In [ ]:
train_features_balanced = balanced_df.drop(['Target'], axis=1).as_matrix()
train_labels_balanced = balanced_df['Target'].as_matrix()

train_features_label = np.insert(train_features_balanced, 0, train_labels_balanced, axis=1)
val_features_label = np.insert(val_features, 0, val_labels, axis=1)
test_features_label = np.insert(test_features, 0, test_labels, axis=1)

np.savetxt("train.csv", train_features_label, delimiter=",")
np.savetxt("validation.csv", val_features_label, delimiter=",")

boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'validation/validation.csv')).upload_file('validation.csv')

s3_input_train = sagemaker.s3_input(s3_data='s3://{}/{}/train'.format(bucket, prefix), content_type='csv')
s3_input_validation = sagemaker.s3_input(s3_data='s3://{}/{}/validation/'.format(bucket, prefix), content_type='csv')

现在，您可以指定一些参数，比如您想要使用的训练实例的类型、数量以及 XGBoost 超参数。一些关键的超参数包括：

- `max_depth`：用于控制算法中每棵树可以构建的深度。树越深，就越能更好地拟合，但计算成本也越大，还可能会导致过拟合。通常，在模型性能方面，您需要在较大数量的浅树和较小数量的深树之间做一些权衡。
- `subsample`：用于控制训练数据的采样。此方法有助于减少过拟合，但如果设置得过低，也可能会让模型无数据可用。
- `num_round`：用于控制提升轮数。这本质上就是使用先前迭代的残差训练的后续模型。同样，轮数越多，对训练数据的拟合就越好，但计算成本也越大，或者会导致过拟合。
- `eta`：用于控制每轮提升的激进程度。值越大，提升越保守。
- `gamma`：用于控制树生长的激进程度。值越大，模型越保守。

In [ ]:
xgb = sagemaker.estimator.Estimator(container,
                                    role,
                                    train_instance_count=1,
                                    train_instance_type='ml.m4.xlarge',
                                    output_path='s3://{}/{}/output'.format(bucket, prefix),
                                    sagemaker_session=sess)
xgb.set_hyperparameters(max_depth=10,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=1,
                        subsample=0.8,
                        silent=0,
                        objective='binary:logistic',
                        eval_metric='auc',
                        num_round=100)

首先，您需要向评估程序指定训练参数。这包括：

- XGBoost 算法容器
- 要使用的 IAM 角色
- 训练实例类型和数量
- 输出数据的 Amazon S3 位置
- 算法超参数

然后是 `.fit()` 函数，它用于指定输出数据的 Amazon S3 位置。在本例中，您传入了训练集和验证集。

In [ ]:
xgb.fit({'train': s3_input_train, 'validation': s3_input_validation})

### 托管

现在您已经使用数据训练了 XGBoost 算法，下面部署一个托管在实时终端节点后面的模型。

在 Amazon SageMaker 上部署您的模型。

In [ ]:
xgb_predictor = # 在此处输入代码

In [ ]:
from sagemaker.predictor import csv_serializer 

def predict_xgboost(model, data, labels, rows=500):
    
    model.content_type = 'text/csv'
    model.serializer = csv_serializer
    model.deserializer = None
    
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = ''
    for array in split_array:
        predictions = ','.join([predictions, model.predict(array).decode('utf-8')])
        
    preds = np.fromstring(predictions[1:], sep=',')
    confusion_matrix = pd.crosstab(index=labels, columns=np.round(preds), rownames=['True'], colnames=['predictions']).astype(int)
    plt.figure(figsize = (5,5))
    sns.heatmap(confusion_matrix, annot=True, fmt='.2f', cmap="YlGnBu").set_title('Confusion Matrix') 

predict_xgboost(xgb_predictor, test_features, test_labels)

In [ ]:
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner

hyperparameter_ranges_xgb = {'eta': ContinuousParameter(0.01, 0.2),
                         'max_depth': IntegerParameter(3, 9),
                         'gamma': IntegerParameter(0, 5),
                         'min_child_weight': IntegerParameter(2, 6),
                         'subsample': ContinuousParameter(0.5, 0.9),
                         'colsample_bytree': ContinuousParameter(0.5, 0.9)}

objective_metric_name_xgb = 'validation:auc'

tuner_xgb = HyperparameterTuner(xgb,
                            objective_metric_name_xgb,
                            hyperparameter_ranges_xgb,
                            max_jobs=10,
                            max_parallel_jobs=1)

tuner_xgb.fit({'train': s3_input_train, 'validation': s3_input_validation}, include_cls_metadata=False)

In [ ]:
client = boto3.Session().client('sagemaker')
tuning_job_result = client.describe_hyper_parameter_tuning_job(HyperParameterTuningJobName=tuner_xgb.latest_tuning_job.job_name)

status = tuning_job_result['HyperParameterTuningJobStatus']
while status != 'Completed':
    print('Reminder: the tuning job has not been completed.')
    
    job_count = tuning_job_result['TrainingJobStatusCounters']['Completed']
    print("%d training jobs have completed" % job_count)
    
    time.sleep(180)

    tuning_job_result = client.describe_hyper_parameter_tuning_job(HyperParameterTuningJobName=tuner_xgb.latest_tuning_job.job_name)
    status = tuning_job_result['HyperParameterTuningJobStatus']
    
print("Training jobs have completed")
is_minimize = (tuning_job_result['HyperParameterTuningJobConfig']['HyperParameterTuningJobObjective']['Type'] != 'Maximize')
objective_name = tuning_job_result['HyperParameterTuningJobConfig']['HyperParameterTuningJobObjective']['MetricName']

在 Amazon SageMaker 上部署优化后的模型。

In [ ]:
xgb_predictor_tuned = tuner_xgb.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

In [ ]:
predict_xgboost(xgb_predictor_tuned, test_features, test_labels)

## 总结

现在，您已经完成了至少两次训练和评估模型的迭代。我们来总结一下该项目，回顾自己学到的内容，并思考可以采取哪些步骤来继续学习（假设您还有时间）。使用下面的单元格回答其中一些问题和其他相关问题：

1.您的模型性能是否能实现您的业务目标？ 如果不能，在有更多时间进行优化的情况下，您希望执行哪些不同的操作？
2.在您对数据集、特征和超参数做出更改之后，模型有多大程度的改进？ 在整个项目中，您认为所采用的哪些技术对模型的改进最大？
3.在整个项目中，您遇到的最大挑战是什么？
4.关于管道的各个方面，有哪些没有解决的问题对您来说是毫无意义的？
5.在完成这个项目的过程中，关于机器学习，您学到的三件最重要的事情是什么？

#### <span style="color: blue;">项目演示：同样在项目演示中总结这些问题的答案。现在，将项目演示期间的所有笔记整理在一起，准备向全班展示您的成果。</span>